In [33]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
#from pymongo import MongoClient
import warnings
import datetime
import csv
import os
import datetime
warnings.filterwarnings(action='ignore')


In [34]:
# 최초 1회 수행 되는 항목
# 복합식 제습기 데이터 호출
dfh_2477 = pd.read_pickle('dfh_H2477_2tank.pkl')

# Timestamp의 형태를 변형, 시간을 1분 단위로 변경
dfh_2477['timestamp'] = pd.to_datetime(dfh_2477['timestamp'])
dfh_2477['timestamp_min'] = dfh_2477['timestamp'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day,dt.hour,1*(dt.minute // 1)))

# 시간 데이터 재 sorting(분류 시 섞이는 경우 발생)  및 중복 시간 데이터 삭제
dfh_2477.sort_values(by=['timestamp_min'], axis = 0, inplace = True,ignore_index = True)
dfh_2477.drop_duplicates(['timestamp_min'], inplace = True, ignore_index = True)

# 향후 추가 분석할 수 있도록 시간을 다양한 형태로 변경 - 사용하지 않음
dfh_2477['daily'] = dfh_2477['timestamp'].apply(lambda x: pd.to_datetime(x.strftime('%Y-%m-%d')))
dfh_2477['weekday'] = dfh_2477['daily'].apply(lambda x: x.weekday())
dfh_2477['hour'] = dfh_2477['timestamp'].apply(lambda x: x.hour)
dfh_2477.sort_values('timestamp', ascending=True, inplace=True)

dfh_2477['Date_str'] = dfh_2477['timestamp'].dt.strftime('%m-%d')
dfh_2477['week'] = dfh_2477['timestamp'].dt.isocalendar().week

In [35]:
dfh_2477 = dfh_2477[dfh_2477['timestamp_min'].dt.date < datetime.date(2019,10,25)]

In [36]:
# 센서 데이터 호출
dfs_2477 = pd.read_pickle('df_H2477_2tank.pkl')

# Timestamp의 형태를 변형, 시간을 1분 단위로 변경
dfs_2477['timestamp'] = pd.to_datetime(dfs_2477['timestamp'])
dfs_2477['timestamp_min'] = dfs_2477['timestamp'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day,dt.hour,1*(dt.minute // 1)))

# 시간 데이터 재 sorting(분류 시 섞이는 경우 발생)  및 중복 시간 데이터 삭제
dfs_2477.sort_values(by=['timestamp_min'], axis = 0, inplace = True,ignore_index = True)
dfs_2477.drop_duplicates(['timestamp_min'], inplace = True, ignore_index = True)

In [37]:
dfs_2477 = dfs_2477[dfs_2477['timestamp_min'].dt.date < datetime.date(2019,10,25)]

In [38]:
dfs_2477 = dfs_2477[['timestamp_min','f3_hum','f10_temp','f10_hum','f3_temp']]
dfh_2477 = dfh_2477[['timestamp_min','exthum','enhum','temp','entemp','extemp','cammode','outstatus_0x0001']]

In [39]:
# 복합식 제습기와 센서 데이터 병합
df_me = pd.merge(dfh_2477, dfs_2477, on = ['timestamp_min'], how = 'left')
# 복합식 제습기와 센서 데이터 병합 후 이름 변경
df_me.rename(columns = {'proj_no_x':'proj_no','tank_x' : 'tank', 'timestamp_x' : 'timestamp', }, inplace = True)

In [40]:
df_me = df_me[df_me['timestamp_min'].dt.date < datetime.date(2019,10,25)]

In [41]:
for col in ['exthum','enhum','temp','entemp','extemp','f3_hum','f10_temp','f10_hum','f3_temp']:
    q1 = df_me[col].quantile(0.25)
    q3 = df_me[col].quantile(0.75)
    iqr = q3-q1

    condition1 = df_me[col] > q3+1.5*iqr
    a = df_me[condition1].index
    condition2 = df_me[col] < q1 - 1.5*iqr
    b = df_me[condition2].index
    df_me = df_me.drop(a)
    df_me = df_me.drop(b)

In [42]:
df_me = pd.merge(dfh_2477['timestamp_min'], df_me, on = ['timestamp_min'], how = 'left')

In [43]:
#센서 전처리 부분(임시)
# 데이터가 없는 부분을 채우는 부분 - 앞 뒤 데이터를 동일 데이터로 입력
df_me.fillna(method='ffill', inplace = True) # 신규 변경 부분
df_me.fillna(method='bfill', inplace = True) # 신규 변경 부분

In [44]:
df_me

,timestamp_min,exthum,enhum,temp,entemp,extemp,cammode,outstatus_0x0001,f3_hum,f10_temp,f10_hum,f3_temp
0,2019-08-21 16:37:00,59.6,85.6,81.5,29.2,27.3,4.0,1.0,57.37,26.77,51.87,25.47
1,2019-08-21 16:38:00,59.4,85.4,107.6,29.2,27.1,4.0,1.0,57.37,26.77,51.87,25.47
2,2019-08-21 16:39:00,57.3,85.4,105.2,29.2,27.3,4.0,1.0,57.27,26.77,51.85,25.50
3,2019-08-21 16:40:00,58.3,85.6,82.5,29.2,27.5,4.0,1.0,57.27,26.77,51.87,25.60
4,2019-08-21 16:41:00,59.5,85.8,76.9,29.2,27.3,4.0,1.0,57.20,26.77,51.85,25.60
...,...,...,...,...,...,...,...,...,...,...,...,...
91647,2019-10-24 23:55:00,52.6,83.1,40.0,19.9,20.8,4.0,0.0,36.72,22.32,40.12,23.62
91648,2019-10-24 23:56:00,52.8,82.8,40.0,19.9,20.8,4.0,0.0,36.72,22.42,40.10,23.62
91649,2019-10-24 23:57:00,52.8,83.0,40.0,19.9,20.8,4.0,0.0,36.85,22.42,40.12,23.62
91650,2019-10-24 23:58:00,52.9,83.0,40.0,19.9,20.8,4.0,0.0,36.72,22.42,39.95,23.62


In [45]:
#OFF 데이터 추가 - 운영 모드 데이터 추가하기 전에 OFF 데이터 추가함.
df_me['new_cammode'] = df_me['cammode'] 
df_me['new_cammode'].loc[df_me['outstatus_0x0001'] == 0] = 0

In [46]:
#SET_MODE One hot encoding

set_mode_status = {
    0: '0', # 전원 OFF
    1: '1', # 송풍모드
    2: '2', # 가온모드
    3: '3', # 송풍+제습
    4: '4', # 냉각
    5: '5', # 냉각+제습
    6: '6', # 가온+제습
}

for i in set_mode_status.keys():
    s = "SET_MODE_" + str(i)
    df_me[s] = 0
    df_me[s].loc[df_me['new_cammode'] == i] = 1

In [47]:
input_feature = ['timestamp_min','f3_hum','f10_hum','f3_temp','f10_temp','exthum','enhum','temp','entemp','extemp','SET_MODE_0','SET_MODE_1','SET_MODE_2','SET_MODE_3','SET_MODE_4','SET_MODE_5','SET_MODE_6']

In [48]:
temp= df_me[input_feature]

In [49]:
df_pre_mode = temp[['SET_MODE_0','SET_MODE_1','SET_MODE_2','SET_MODE_3','SET_MODE_4','SET_MODE_5','SET_MODE_6']]

In [50]:
df_mode_t = pd.concat([df_pre_mode[10:],df_pre_mode[-10:]],ignore_index=True)
df_mode_t = df_mode_t.set_index(temp.index)

In [52]:
temp[['SET_MODE_0','SET_MODE_1','SET_MODE_2','SET_MODE_3','SET_MODE_4','SET_MODE_5','SET_MODE_6']] = df_mode_t

In [53]:
temp = temp[temp.columns[:-1]]

In [54]:
temp.columns = ['timestamp_min','f3_hum','f10_hum','f3_temp','f10_temp','exthum','enhum','temp','entemp','extemp','F_SET_MODE_0','F_SET_MODE_1','F_SET_MODE_2','F_SET_MODE_3','F_SET_MODE_4','F_SET_MODE_5']

In [55]:
temp

,timestamp_min,f3_hum,f10_hum,f3_temp,f10_temp,exthum,enhum,temp,entemp,extemp,F_SET_MODE_0,F_SET_MODE_1,F_SET_MODE_2,F_SET_MODE_3,F_SET_MODE_4,F_SET_MODE_5
0,2019-08-21 16:37:00,57.37,51.87,25.47,26.77,59.6,85.6,81.5,29.2,27.3,0,0,0,0,1,0
1,2019-08-21 16:38:00,57.37,51.87,25.47,26.77,59.4,85.4,107.6,29.2,27.1,0,0,0,0,1,0
2,2019-08-21 16:39:00,57.27,51.85,25.50,26.77,57.3,85.4,105.2,29.2,27.3,0,0,0,0,1,0
3,2019-08-21 16:40:00,57.27,51.87,25.60,26.77,58.3,85.6,82.5,29.2,27.5,0,0,0,0,1,0
4,2019-08-21 16:41:00,57.20,51.85,25.60,26.77,59.5,85.8,76.9,29.2,27.3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91647,2019-10-24 23:55:00,36.72,40.12,23.62,22.32,52.6,83.1,40.0,19.9,20.8,1,0,0,0,0,0
91648,2019-10-24 23:56:00,36.72,40.10,23.62,22.42,52.8,82.8,40.0,19.9,20.8,1,0,0,0,0,0
91649,2019-10-24 23:57:00,36.85,40.12,23.62,22.42,52.8,83.0,40.0,19.9,20.8,1,0,0,0,0,0
91650,2019-10-24 23:58:00,36.72,39.95,23.62,22.42,52.9,83.0,40.0,19.9,20.8,1,0,0,0,0,0


In [275]:
temp.to_feather('./ecm_input.ftr')